In [1]:
pip install nibabel

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import nibabel as nib
from ants import image_read

In [3]:
import os
import subprocess
import matplotlib.pyplot as plt
from multiprocessing import Pool, cpu_count

FSL_BIN_DIR = "/data/csc4801/WangLab/y/share/fsl/bin/"
FSLREORIENT_PATH = os.path.join(FSL_BIN_DIR, "fslreorient2std")
FLIRT_PATH = os.path.join(FSL_BIN_DIR, "flirt")


def plot_middle(data, slice_no=None):
    if not slice_no:
        slice_no = data.shape[-1] // 2
    plt.figure()
    plt.imshow(data[..., slice_no], cmap="gray")
    plt.show()
    return


def registration(src_path, dst_path, ref_path):
    # Use absolute paths for the executables
    command = [FLIRT_PATH, "-in", src_path, "-ref", ref_path, "-out", dst_path,
               "-bins", "256", "-cost", "corratio", "-searchrx", "0", "0",
               "-searchry", "0", "0", "-searchrz", "0", "0", "-dof", "12",
               "-interp", "spline"]
    subprocess.call(command, stdout=open(os.devnull, "r"),
                    stderr=subprocess.STDOUT)
    return

def orient2std(src_path, dst_path):
    # Use absolute path for fslreorient2std
    command = [FSLREORIENT_PATH, src_path, dst_path]
    subprocess.call(command)
    return


def create_dir(path):
    if not os.path.isdir(path):
        os.makedirs(path)
    return


def unwarp_main(arg, **kwarg):
    return registration(*arg, **kwarg)


def registration1(src_path, dst_path, ref_path):
    print("Registration on: ", src_path)
    try:
        orient2std(src_path, dst_path)
        registration(dst_path, dst_path, ref_path)
    except RuntimeError:
        print("\tFailed on: ", src_path)
    return


def reg_main(image_path, output_dir, ref_path):
    # Prepare destination path
    filename = os.path.basename(image_path)
    dest_path = os.path.join(output_dir, filename)
    create_dir(output_dir)
    # Perform registration
    registration1(image_path, dest_path, ref_path)

In [4]:
from __future__ import print_function
import os
import subprocess
import nibabel as nib
import numpy as np
import zipfile

FSL_BIN_DIR = "/data/csc4801/WangLab/y/share/fsl/bin/"
BET_PATH = os.path.join(FSL_BIN_DIR, "bet")


def create_dir(path):
    if not os.path.isdir(path):
        os.makedirs(path)


def bet(src_path, dst_path, frac="0.5"):
    command = [BET_PATH, src_path, dst_path, "-R", "-f", frac, "-g", "0"]
    subprocess.call(command)
    return


def unwarp_strip_skull(arg, **kwarg):
    return strip_skull(*arg, **kwarg)


def strip_skull(src_path, dst_path, frac="0.4"):
    print("Working on :", src_path)
    try:
        bet(src_path, dst_path, frac)
    except RuntimeError:
        print("\tFailed on: ", src_path)
    return


def ss_main(image_path, output_dir):

    if not os.path.exists(image_path):
        print("Error: Image path does not exist.")
        return
    if not os.path.isdir(output_dir):
        print("Error: Output directory does not exist.")
        return

    dest_filename = os.path.basename(image_path)
    dest_path = os.path.join(output_dir, dest_filename)

    if dest_path.endswith('.gz'):
        dest_path = dest_path[:-3]

    create_dir(dest_path)
    print(dest_path)
    strip_skull(image_path, dest_path)

    processed_img = nib.load(dest_path + ".gz")
    data = processed_img.get_fdata()
    output_array = np.array(data)

    return output_array

In [5]:
def preprocess_main(reg_input_image_path, reg_dir, skull_strip_image_path, skull_strip_dir, ref_path):
    try:
        reg_main(reg_input_image_path, reg_dir, ref_path)
    except Exception as e:
        print(f"An error occurred in registration: {e}")

    try:
        output_array = ss_main(skull_strip_image_path, skull_strip_dir)
        return output_array
    except Exception as e:
        print(f"An error occurred in skull stripping: {e}")
        return None

In [6]:
import numpy as np
from ants import threshold_image, image_read, resample_image_to_target, registration, from_numpy
import nibabel as nib

from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pearsonr

class Scan:

    path: str
    data: np.array
    clean_data: np.array
    rois: np.array

    GMVs: np.array
    R2F: np.array
    R2SN: np.array
    RMCS: np.array

    def __init__(self, path):
        self.path = path
        self.scan()

        self.GMVs = np.zeros((246, 1))
        self.R2F = np.zeros((246, 47))
        self.R2SN = np.zeros((246, 246))
        self.RMCS = np.zeros((246, 1))
        
    def scan(self):
        img = nib.load(self.path)
        self.data = np.array(img.dataobj)
        self.preprocess()


    def preprocess(self):
        reg_dir = "/data/csc4801/WangLab/cleaned_scans/registered"
        skull_strip_dir = "/data/csc4801/WangLab/cleaned_scans/skullstripped"
        ref_path = "/data/csc4801/WangLab/MNI152_T1_1mm.nii.gz"
        reg_input_image_path = self.path
        skull_strip_image_path = self.path  # Adjust as necessary for your workflow

        self.clean_data = preprocess_main(reg_input_image_path, reg_dir, skull_strip_image_path, skull_strip_dir, ref_path)
        if self.clean_data is None:
            self.clean_data = self.data  # Fallback to original data if preprocessing fails

    
    def split_rois(self, atlas, ref, threshold=50):
        """
        Given an MR scan and atlas, split into regions of interest defined by the atlas.

        Current supported atlases:
        - Brainnetome (atlas.brainnetome.org)

        Params:
        scan: Numpy.memmap or similar object | ANTsImage
            The scan to split into ROIs
        atlas: str
            Which atlas to use to perform splitting (see list of supported atlases)
        threshold: int
            The threshold to use for the atlas' probabilistic map to determine which voxels
            in the scan are included in each region
        """
        scan = from_numpy(self.clean_data)
        # register scan to atlas reference
        resampled = resample_image_to_target(image=scan, target=ref)
        registered_scan = registration(fixed=ref, moving=resampled, type_of_transform='SyN')['warpedmovout'].numpy()

        at = (atlas > threshold).astype('float')
        rois = []
        for i in range(at.shape[-1]):
            atlas_roi = at[:,:,:,i]
            roi = registered_scan * atlas_roi
            rois.append(roi)
        self.rois = np.array(rois)

In [7]:
try:
    atlases = {
        "Brainnetome": nib.load("/data/csc4801/WangLab/BNA_PM_4D.nii").get_fdata()
    }

    atlas_refs = {
        "Brainnetome": image_read("/data/csc4801/WangLab/atlas_reference/HCP40_MNI_1.25mm.nii")
    }
except FileNotFoundError:
    print("WARNING: Atlas and Atlas reference were not in the expected locations. Please use set_atlas() to set custom atlas paths.")
    atlases = {}
    atlas_refs = {}

In [8]:
def set_atlas(atlas, atlas_loc, ref_loc):
    print("Setting atlases")
    atlases[atlas] = nib.load(atlas_loc).get_fdata()
    atlas_refs[atlas] = image_read(ref_loc)
    print("Atlas loaded")

In [9]:
def save_rois(rois, patient_id, output_directory):
    output_dir = os.path.join(output_directory, patient_id)
    os.makedirs(output_dir, exist_ok=True)
    for i, roi in enumerate(rois):
        roi_img = nib.Nifti1Image(roi, np.eye(4))  # Assuming affine as identity for simplicity
        nib.save(roi_img, os.path.join(output_dir, f"roi_{i}.nii"))
    print("Saved ROIs")

In [10]:
def process_scans(directory, output_directory, atlas="Brainnetome", threshold=50):
    print("Preprocessing Scans")
    for patient_id in os.listdir(directory):
        patient_path = os.path.join(directory, patient_id)
        if os.path.isdir(patient_path):
            for filename in os.listdir(patient_path):
                if filename.endswith(".nii"):
                    # print(filename)
                    scan_path = os.path.join(patient_path, filename)
                    scan = Scan(scan_path)
                    scan.split_rois(atlases[atlas], atlas_refs[atlas], threshold)
                    save_rois(scan.rois, patient_id, output_directory)

In [11]:
def process_scans2(directory, output_directory, atlas="Brainnetome", threshold=50):
    print("Preprocessing Scans")
    for patient_id in os.listdir(directory):
        patient_path = os.path.join(directory, patient_id)
        if os.path.isdir(patient_path):
            for root, dirs, files in os.walk(patient_path):
                if files:
                    filename = files[-1]
                    scan_path = os.path.join(patient_path, filename)
                    scan = Scan(scan_path)
                    scan.split_rois(atlases[atlas], atlas_refs[atlas], threshold)
                    save_rois(scan.rois, patient_id, output_directory)

In [12]:
def main():
    print("running main")
    set_atlas("Brainnetome", "/data/csc4801/WangLab/BNA_PM_4D.nii", "/data/csc4801/WangLab/atlas_reference/HCP40_MNI_1.25mm.nii")
    process_scans2("/data/csc4801/WangLab/training/AD", "/data/csc4801/WangLab/cleaned_scans")

In [13]:
main()

running main
Setting atlases
Atlas loaded
Preprocessing Scans
Registration on:  /data/csc4801/WangLab/training/AD/136_S_0300/ADNI_136_S_0300_MR_MPR____N3__Scaled_Br_20070809225810386_S22443_I66754.nii
An error occurred in registration: Fixed and moving images must be ANTsImage objects
/data/csc4801/WangLab/cleaned_scans/skullstripped/ADNI_136_S_0300_MR_MPR____N3__Scaled_Br_20070809225810386_S22443_I66754.nii
Working on : /data/csc4801/WangLab/training/AD/136_S_0300/ADNI_136_S_0300_MR_MPR____N3__Scaled_Br_20070809225810386_S22443_I66754.nii
Saved ROIs
Registration on:  /data/csc4801/WangLab/training/AD/023_S_0916/ADNI_023_S_0916_MR_MPR__GradWarp__B1_Correction__N3__Scaled_2_Br_20081001154612955_S30418_I118884.nii
An error occurred in registration: Fixed and moving images must be ANTsImage objects
/data/csc4801/WangLab/cleaned_scans/skullstripped/ADNI_023_S_0916_MR_MPR__GradWarp__B1_Correction__N3__Scaled_2_Br_20081001154612955_S30418_I118884.nii
Working on : /data/csc4801/WangLab/traini